# CMVD PRS Evaluation

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import linregress
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve, f1_score, \
    balanced_accuracy_score, average_precision_score, precision_score, recall_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

import math
import seaborn as sns

## PRS Eval - Models 1,2,3

In [ ]:
prs = pd.read_csv(f'', sep = '\t')
prs

In [ ]:
phenotype = pd.read_csv("")
samps = list(phenotype['PMBB_ID'])
phenotype = phenotype.set_index("")
prs = pd.read_csv(f'', sep = '\t')
prs = prs[prs['IID'].isin(samps)]
prs = prs.set_index("IID")
prs.index = [str(x) for x in prs.index]
prs_pheno = prs.join(phenotype)

# feature_cols = ['Enrollment_age', 'gender_source_value', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5'] #- JUST covs
feature_cols = ['SCORE1_AVG'] #- JUST PRS
# feature_cols = ['SCORE1_AVG', 'Enrollment_age', 'gender_source_value', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5'] #- all covs

prs_pheno_fin = prs_pheno[feature_cols + ['CMVD']].dropna()
X = prs_pheno_fin[feature_cols] 
y = prs_pheno_fin['CMVD'] 

results = []

for j in range(0,100):
    # split X and y into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=j+1)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    logreg = LogisticRegressionCV(
    penalty='l1',
    solver='saga',
    cv=5,
    scoring='roc_auc',
    max_iter=5000,          
    n_jobs=-1
)

    # fit
    logreg.fit(X_train_scaled, y_train)
    y_pred = logreg.predict(X_test_scaled)
    y_pred_proba = logreg.predict_proba(X_test_scaled)[::,1]
    auc = metrics.roc_auc_score(y_test, y_pred_proba)
    mcc = matthews_corrcoef(y_test, y_pred)

    test_accuracy = accuracy_score(y_test, y_pred)
    test_roc_auc = roc_auc_score(y_test, y_pred_proba)
    test_f1 = f1_score(y_test, y_pred)
    test_balanced_acc = balanced_accuracy_score(y_test, y_pred)
    test_auprc = average_precision_score(y_test, y_pred_proba)
    test_conf_matrix = confusion_matrix(y_test, y_pred)
    test_class_report = classification_report(y_test, y_pred, output_dict=True)
    test_precision = precision_score(y_test, y_pred)
    test_recall = recall_score(y_test, y_pred)

    results.append({
            'iteration': j,
            'test_accuracy': test_accuracy,
            'test_roc_auc': test_roc_auc,
            'test_f1': test_f1,
            'test_balanced_acc': test_balanced_acc,
            'test_auprc': test_auprc,
            'test_precision': test_precision,
            'test_recall': test_recall,
            'test_mcc': mcc})

df = pd.DataFrame(results)
df

In [ ]:
sumstats = df.drop(columns = ['iteration'])[['test_roc_auc', 'test_auprc', 'test_balanced_acc',  'test_f1', 'test_mcc']]
mean_std = sumstats.agg(['mean', 'count', 'std']).T
mean_std

ci95_hi = []
ci95_lo = []

for i in mean_std.index:
    m, c, s = mean_std.loc[i]
    ci95_hi.append(m + 1.96*s/math.sqrt(c))
    ci95_lo.append(m - 1.96*s/math.sqrt(c))

mean_std['ci95_lo'] = ci95_lo
mean_std['ci95_hi'] = ci95_hi
print(mean_std.round(3))

In [ ]:
phenotype = pd.read_csv("")
samps = list(phenotype['PMBB_ID'])
phenotype = phenotype.set_index("")
phenotype['CMVD'] = phenotype['CMVD']
prs = pd.read_csv(f'', sep = '\t')
prs = prs[prs['IID'].isin(samps)]
prs = prs.set_index("IID")
prs.index = [str(x) for x in prs.index]
prs_pheno = prs.join(phenotype)

prot = pd.read_csv('', index_col = 'PMBB_ID')
demo_cols = ['Enrollment_age', 'gender_source_value', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5'] + list(prot.columns)

prs_pheno_mod = prs_pheno[demo_cols + list(prot.columns)+ ['SCORE1_AVG'] + ['CMVD']].dropna()
prs_pheno_mod.to_csv('')
prs_pheno_mod

### Model Including PRS as a Feature

In [ ]:
# %load elasticnet_logistic_classification.py

In [ ]:
prs_pheno_mod = pd.read_csv('')
prot = pd.read_csv('', index_col = 'PMBB_ID')

demo_cols = ['Enrollment_age', 'gender_source_value', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5']
X = prs_pheno_mod[demo_cols + ['SCORE1_AVG']]
X = X.loc[:, ~X.columns.duplicated()]
y = prs_pheno_mod["CMVD"]

results = run_elasticnet_classification(X, y, n_iter=20, output_prefix='')

In [ ]:
# %load xgboost_classification_bestshap.py

In [ ]:
prs_pheno_mod = pd.read_csv('')
prot = pd.read_csv('', index_col = 'PMBB_ID')

demo_cols = ['Enrollment_age', 'gender_source_value', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5']
# X = prs_pheno_mod[demo_cols + list(prot.columns) + ['SCORE1_AVG']]
X = prs_pheno_mod[demo_cols + ['SCORE1_AVG']]
X = X.loc[:, ~X.columns.duplicated()]
y = prs_pheno_mod["CMVD"]

results = run_xgboost_classification(X, y, n_iter=10, output_prefix='', n_trials = 10)

In [ ]:
# %load autoencoder_classification_bestshap.py

In [ ]:
prs_pheno_mod = pd.read_csv('')
prot = pd.read_csv('', index_col = 'PMBB_ID')

demo_cols = ['Enrollment_age', 'gender_source_value', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5']
# X = prs_pheno_mod[demo_cols + list(prot.columns) + ['SCORE1_AVG']]
X = prs_pheno_mod[demo_cols + ['SCORE1_AVG']]
X = X.loc[:, ~X.columns.duplicated()].dropna()
y = prs_pheno_mod["CMVD"]

results = run_autoencoder_classification(X, y, n_iter=10, output_dir='', n_trials = 10)

In [ ]:
# %load DL_classification_bestshap.py

In [ ]:
prs_pheno_mod = pd.read_csv('')
prot = pd.read_csv('', index_col = 'PMBB_ID')

demo_cols = ['Enrollment_age', 'gender_source_value', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5']
# X = prs_pheno_mod[demo_cols + list(prot.columns) + ['SCORE1_AVG']]
X = prs_pheno_mod[demo_cols + ['SCORE1_AVG']]
X = X.loc[:, ~X.columns.duplicated()]
y = prs_pheno_mod["CMVD"]

results = run_pytorch_classification(X, y, n_iter=10, output_prefix='', n_trials = 10)